In [1]:
from dotenv import load_dotenv
from glob import glob
from pprint import pprint
import os

In [2]:
load_dotenv()

True

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model='text-embedding-3-small'
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name='chroma_test',
    persist_directory='./chroma_db'
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [4]:

print(f'벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}')

벡터 저장소에 저장된 문서 수: 0


In [8]:
# Prompt + LLM
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.3,
    max_tokens=100,
)

messages = [
    ('system', 'You are a helped assistant.'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages)

print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helped assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [9]:
prompt_text = prompt.format(query='테슬라 창업자는 누구인가요?')

print(prompt_text)

System: You are a helped assistant.
Human: 테슬라 창업자는 누구인가요?


In [10]:
response = llm.invoke(prompt_text)

print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey B. Straubel) 등 여러 명이 있습니다. 하지만 엘론 머스크는 테슬라의 CEO로서 가장 널리 알려져 있으며, 회사의 비전과 방향성을 이끌어온 주요 인물입니다. 테


In [11]:
# LCEL 체인을 구성
chain = prompt | llm

In [12]:
response = chain.invoke({'query': '테슬라 창업자는 누구인가요?'})

print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)가 아니지만, 그는 회사의 초기 투자자이자 CEO로 잘 알려져 있습니다. 테슬라는 2003년에 마틴 에버하르드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하면서 회사의 주요 인물로 자리잡게 되었습니다.


In [ ]:
#query가 하나밖에 없어서

response = chain.invoke('테슬라 창업자는 누구인가요?')

print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 발전에 중요한 역할을 하게 되었습니다. 따라서 엘론 머스크는 테슬


In [14]:
# prompt + LLM + Output parser
response

AIMessage(content='테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 발전에 중요한 역할을 하게 되었습니다. 따라서 엘론 머스크는 테슬', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 27, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'length', 'logprobs': None}, id='run-5ff8af6c-6af9-401d-a5b1-cc5b85e94be7-0', usage_metadata={'input_tokens': 27, 'output_tokens': 100, 'total_tokens': 127})

In [15]:
# 문자열 파서
from langchain_core.output_parsers import StrOutputParser

# 출력 파서를 새서
output_parser = StrOutputParser()
output_parser.invoke(response)

'테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 발전에 중요한 역할을 하게 되었습니다. 따라서 엘론 머스크는 테슬'

In [17]:
str_chain = prompt | llm | output_parser

query = '리비안의 설립년도는 언제인가요?'
str_response = str_chain.invoke(query)

print(str_response)

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 기업으로, 특히 전기 픽업트럭과 SUV 모델에 주력하고 있습니다.


In [21]:
# Json parser
from langchain_core.output_parsers import JsonOutputParser

Json_parser = JsonOutputParser()

json_response = chain.invoke('테슬라 창업자는 누구인가요? Json 형식으로 출력해 주세요')

print(json_response)

json_parser_output = Json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "창업자": "엘론 머스크",\n  "설명": "테슬라는 2003년에 마틴 에버하드와 마크 타페닝에 의해 설립되었지만, 엘론 머스크는 2004년에 투자자로 참여하여 CEO로 취임하며 회사의 성장에 큰 영향을 미쳤습니다."\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 34, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None} id='run-34c133ed-7ee1-460f-9e9b-974d4bc5f42f-0' usage_metadata={'input_tokens': 34, 'output_tokens': 80, 'total_tokens': 114}
{'창업자': '엘론 머스크', '설명': '테슬라는 2003년에 마틴 에버하드와 마크 타페닝에 의해 설립되었지만, 엘론 머스크는 2004년에 투자자로 참여하여 CEO로 취임하며 회사의 성장에 큰 영향을 미쳤습니다.'}


In [ ]:
# Pydatic 모델을 생성
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    '''Information about a person.'''
    name: str = Field(..., description='The name of the person')
    title: str = Field(..., description='The title or position of the person')

person_parser = PydanticOutputParser(pydantic_object=Person)

print('==================================================')
print(person_parser.get_format_instructions())
print('==================================================')

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "properties": {"name": {"title": "Name", "description": "The name of the person", "type": "string"}, "title": {"title": "Title", "description": "The title or position of the person", "type": "string"}}, "required": ["name", "title"]}
```


In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
    ('system', 'Answer the user query, Wrap the output in "json" tags\n {format_instructions}'),
    ('human', '{query}')
    ]
).partial(format_instructions=person_parser.get_format_instructions())

In [24]:
person_chain = prompt | llm | person_parser
response = person_chain.invoke('테슬라 창업자는 누구인가요?')
response

Person(name='Elon Musk', title='CEO and Founder of Tesla, Inc.')

In [25]:
# chat Completion Methods
# stream
import time

for chunk in llm.stream('테슬라 창업자는 누구인가요?'):
    print(chunk.content, end='', flush=True)
    time.sleep(0.1)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하면서 CEO로 취임하게 되었고, 회사의 비전과 방향성을 크게 변화시켰습니다. 현재 엘론 머스크

In [26]:
# batch
questions = [
    '테슬라의 창업자는 누구인가요?',
    '리비안의 창업자는 누구인가요?'
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프레처(JB Straubel), 이안 라이트(Ian Wright) 등 여러 명입니다. 그러나 엘론 머스크는 테슬라의 CEO로서 가장 잘 알려져 있으며, 회사의 성장과 발전에 큰 영향을 미쳤습니다. 테슬라는

================================== Ai Message ==================================

리비안(Rivian)의 창업자는 RJ 스케링(RJ Scaringe)입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 제조업체로서 SUV와 픽업트럭을 주로 생산하고 있습니다. 리비안은 지속 가능한 이동 수단을 목표로 하고 있으며, 전기차 시장에서 주목받고 있는 기업 중 하나입니다.



In [28]:
# 문서 검색기 (retriever)
retriever = vectorstore.as_retriever(
    search_kwargs={'k':2}
)

query = '테슬라 창업자는 누구인가요?'
retrieved_docs = retriever.invoke(query)

retrieved_docs_text = '\n'.join([doc.page_content for doc in retrieved_docs])

pprint(retrieved_docs_text)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [29]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

runnable = RunnableParallel(
    {
        'context': itemgetter('context'),
        'question': itemgetter('question')
    }
)

response = runnable.invoke({'context': retrieved_docs_text, 'question':query})

response

{'context': '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.',
 'question': '테슬라 창업자는 누구인가요?'}

In [33]:
# RunnablePassthrough,, RunnableLambda
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def count_num_words(text):
    return len(text.split())

runnable = RunnableParallel(
    question = RunnablePassthrough(),
    word_count = RunnableLambda(count_num_words)
)

runnable.invoke('테슬라 창업자는 누구인가요?')

{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

In [34]:
# RAG 프롬프트 템플릿 파이프라인 구성
from langchain.prompts import ChatPromptTemplate

template = '''다음 맥락만을 바탕으로 질문에 답하세요.
외부 정보나 지식을 사용하지 마세요.
답변이 문맥에 맞지 않는 경우 답변 '잘 모르겠습니다.'

[Context]
{context}

[Question]
{question}

[Answer]
'''

prompt = ChatPromptTemplate.from_template(template)

prompt.pretty_print()

================================ Human Message =================================

다음 맥락만을 바탕으로 질문에 답하세요.
외부 정보나 지식을 사용하지 마세요.
답변이 문맥에 맞지 않는 경우 답변 '잘 모르겠습니다.'

[Context]
{context}

[Question]
{question}

[Answer]



In [35]:
retrever = vectorstore.as_retriever(search_kwargs={'k':2})

# 문서 포맷터 함수
def format_docs(docs):
    return '\n\n'.join([doc.page_content for doc in docs])

retriever_chain = retriever | format_docs

response = retriever_chain.invoke('테슬라 창업자는 누구인가요?')

pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import  ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0, max_tokens=100)

rag_chain = (
    {'context': retriever_chain, 'question':RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = '테슬라 창업자는 누구인가요?'
response = rag_chain.invoke(query)

In [37]:
response

'테슬라의 창업자는 마틴 에버하드와 마크 타페닝입니다.'